## Overview

This analysis involves running the REopt API with two distinct scenarios:
- **Scenario 1: PV only** - Incorporates only a photovoltaic (PV) system.
- **Scenario 2: PV + BESS** - Includes both a PV system and a Battery Energy Storage System (BESS).

The API responses are stored as JSON files in the outputs folder for subsequent processing.


## Note:
Run this section of code first (before running the rest) and copy the Run UUIDs provided at the end as inputs within the main script as `run_uuid_1` and `run_uuid_2`

In [7]:
import pandas as pd
import json
import requests
import time
import os
import uuid
from collections import defaultdict
import re

API_KEY = "gYV8t7d6c5naotp67meIJyJRi6DksKv0VfPSQzEa"  # Replace with your API key
# Define the API key and URL
root_url = "https://developer.nrel.gov/api/reopt/stable"
# following is not necessary but silences warnings:
# InsecureRequestWarning: Unverified HTTPS request is being made to host 'developer.nrel.gov'. Adding certificate verification is strongly advised.
import urllib3
urllib3.disable_warnings()

Code to run two scenarios to get run_uuid for results for camparison

In [8]:
# inputs_path = os.path.join(".", 'inputs')
# outputs_path = os.path.join(".", 'outputs')

# # Define the post data for both scenarios
# post_1 = {
#     "Settings": {
#       "run_bau": True
#     },
#     "Site": {
#         "longitude": -118.1164613,
#         "latitude": 34.5794343
#     },
#     "PV": {
#         "array_type": 0
#     },
#     "ElectricLoad": {
#         "doe_reference_name": "RetailStore",
#         "annual_kwh": 100000.0,
#         "year": 2017
#     },
#     "ElectricTariff": {
#         "urdb_label": "5ed6c1a15457a3367add15ae"
#     },
#     "Financial": {
#         "elec_cost_escalation_rate_fraction": 0.026,
#         "owner_discount_rate_fraction": 0.081,
#         "analysis_years": 20,
#         "offtaker_tax_rate_fraction": 0.4,
#         "owner_tax_rate_fraction": 0.4,
#         "om_cost_escalation_rate_fraction": 0.025
#     }
# }

# post_2 = {
#     "Settings": {
#         "run_bau": True
#     },
#     "Site": {
#         "longitude": -118.1164613,
#         "latitude": 34.5794343
#     },
#     "PV": {
#         "array_type": 0
#     },
#     "ElectricLoad": {
#         "doe_reference_name": "RetailStore",
#         "annual_kwh": 100000.0,
#         "year": 2017
#     },
#     "ElectricStorage":{
#     },
#     "ElectricTariff": {
#         "urdb_label": "5ed6c1a15457a3367add15ae"
#     },
#     "Financial": {
#         "elec_cost_escalation_rate_fraction": 0.026,
#         "owner_discount_rate_fraction": 0.081,
#         "analysis_years": 20,
#         "offtaker_tax_rate_fraction": 0.4,
#         "owner_tax_rate_fraction": 0.4,
#         "om_cost_escalation_rate_fraction": 0.025
#     }
# }

# # Function to POST inputs to the API
# def post_inputs_to_api(post_data, api_key, api_url):
#     headers = {'x-api-key': api_key}
#     response = requests.post(f"{api_url}/job/", headers=headers, json=post_data, verify=False)
#     response.raise_for_status()
#     return response.json()["run_uuid"]

# # Function to poll for results
# def poll_for_results(run_uuid, api_key, api_url, interval=5):
#     headers = {'x-api-key': api_key}
#     while True:
#         response = requests.get(f"{api_url}/job/{run_uuid}/results/", headers=headers, verify=False)
#         response.raise_for_status()
#         result = response.json()
#         if result["status"] != "Optimizing...":
#             return result
#         time.sleep(interval)



# # Post the inputs to the API and get the run_uuid for both scenarios
# run_uuid_1 = post_inputs_to_api(post_1, API_KEY, root_url)
# print(f"Run UUID for Scenario 1: {run_uuid_1}")

# run_uuid_2 = post_inputs_to_api(post_2, API_KEY, root_url)
# print(f"Run UUID for Scenario 2: {run_uuid_2}")

# # Poll for results for both scenarios
# results_1 = poll_for_results(run_uuid_1, API_KEY, root_url)
# print("Results retrieved successfully for Scenario 1.")

# results_2 = poll_for_results(run_uuid_2, API_KEY, root_url)
# print("Results retrieved successfully for Scenario 2.")

# # Save the results to JSON files
# outputs_file_name_1 = f"response_{run_uuid_1}.json"
# with open(os.path.join(outputs_path,outputs_file_name_1), 'w') as fp:
#     json.dump(results_1, fp, indent=4)
# print(f"Results saved to {outputs_file_name_1}")

# outputs_file_name_2 = f"response_{run_uuid_2}.json"
# with open(os.path.join(outputs_path,outputs_file_name_2), 'w') as fp:
#     json.dump(results_2, fp, indent=4)
# print(f"Results saved to {outputs_file_name_2}")


## Key Functions and Steps in the Analysis

### 1. `get_scenario_results`
Retrieves the results from the REopt API for a specified scenario using its unique run UUID. This function:
- Sends a GET request to the REopt API.
- Fetches the results in JSON format.

### 2. `process_scenarios`
Processes multiple scenarios to generate a consolidated DataFrame. Key steps include:
- **Data Extraction**: Utilizes the `get_REopt_data` function with a specified `config` to extract relevant data points from the API responses.
- **BAU Data Handling**: Checks and manages Business-As-Usual (BAU) values to ensure data consistency across scenarios.
- **Final DataFrame Creation**: Combines data from all scenarios into a single DataFrame and processes it for final output.

### 3. `get_REopt_data`
Extracts specific data points from the API response JSON based on a configuration (`config`). This function:
- Flattens the nested JSON structure.
- Retrieves values according to the configuration settings.
- Cleans and structures the data into a pandas DataFrame.

### 4. `generate_data_dict`
Generates a data dictionary based on the provided configuration and flattened data. This dictionary is then converted into a DataFrame for further analysis.


In [9]:
import xlsxwriter
import pandas as pd
import requests
import json
from collections import defaultdict
import re
import uuid

#### Helper Functions
def get_with_suffix(df, key, suffix, default_val=0):
    """Fetch value from dataframe with an optional retriaval of _bau suffix."""
    if not key.endswith("_bau"):
        key = f"{key}{suffix}"
    return df.get(key, default_val)

def flatten_dict(d, parent_key='', sep='.'):
    """Flatten nested dictionary."""
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

def clean_data_dict(data_dict):
    """Clean data dictionary by removing default values."""
    for key, value_array in data_dict.items():
        new_value_array = [
            "" if v in [0, float("nan"), "NaN", "0", "0.0", "$0.0", -0, "-0", "-0.0", "-$0.0", None] else v
            for v in value_array
        ]
        data_dict[key] = new_value_array
    return data_dict

def sum_vectors(data):
    """Sum numerical vectors within a nested data structure."""
    if isinstance(data, dict):
        return {key: sum_vectors(value) for key, value in data.items()}
    elif isinstance(data, list):
        if all(isinstance(item, (int, float)) for item in data):
            return sum(data)
        else:
            return [sum_vectors(item) for item in data]
    else:
        return data
    
#### Core Functions
def generate_data_dict(config, df_gen, suffix):
    """Generate data dictionary based on configuration and dataframe."""
    data_dict = defaultdict(list)
    for var_key, col_name in config:
        if callable(var_key):
            val = var_key(df_gen)
        else:
            val = get_with_suffix(df_gen, var_key, suffix, "-")
        data_dict[col_name].append(val)
    return data_dict

def get_REopt_data(data_f, scenario_name, config):
    """Fetch and format data for a specific REopt scenario."""
    scenario_name_str = str(scenario_name)
    suffix = "_bau" if re.search(r"(?i)\bBAU\b", scenario_name_str) else ""
    
    df_gen = flatten_dict(data_f)
    data_dict = generate_data_dict(config, df_gen, suffix)
    data_dict["Scenario"] = [scenario_name_str]

    col_order = ["Scenario"] + [col_name for _, col_name in config]
    df_res = pd.DataFrame(data_dict)
    df_res = df_res[col_order]

    return df_res

def get_bau_values(mock_scenarios, config):
    """Retrieve BAU values for comparison."""
    bau_values = {col_name: None for _, col_name in config}
    for scenario in mock_scenarios:
        df_gen = flatten_dict(scenario["outputs"])
        for var_key, col_name in config:
            try:
                key = var_key.__code__.co_consts[1]
            except IndexError:
                print(f"Warning: Could not find constant in lambda for {col_name}. Skipping...")
                continue

            key_bau = f"{key}_bau"
            if key_bau in df_gen:
                value = df_gen[key_bau]
                if bau_values[col_name] is None:
                    bau_values[col_name] = value
                elif bau_values[col_name] != value:
                    raise ValueError(f"Inconsistent BAU values for {col_name}. This should only be used for portfolio cases with the same Site, ElectricLoad, and ElectricTariff for energy consumption and energy costs.")
    return bau_values

def get_scenario_results(run_uuid):
    """Retrieve scenario results from an external API."""
    results_url = f"{root_url}/job/{run_uuid}/results/?api_key={API_KEY}"
    response = requests.get(results_url, verify=False)
    response.raise_for_status()
    result_data = response.json()
    
    processed_data = sum_vectors(result_data)
    
    ### outputs json with the simplified REopt results where vectors are summed for a single value
    # with open(f"{run_uuid}.json", "w") as json_file:
    #     json.dump(processed_data, json_file, indent=4)
    
    return processed_data

def process_scenarios(scenarios):
    """Process multiple scenarios and generate a combined dataframe."""
    config = [
        (lambda df: get_with_suffix(df, "PV.size_kw", ""), "PV Size (kW)"),
        (lambda df: get_with_suffix(df, "Wind.size_kw", ""), "Wind Size (kW)"),
        (lambda df: get_with_suffix(df, "CHP.size_kw", ""), "CHP Size (kW)"),
        (lambda df: get_with_suffix(df, "PV.annual_energy_produced_kwh", ""), "PV Total Electricity Produced (kWh)"),
        (lambda df: get_with_suffix(df, "PV.electric_to_grid_series_kw", ""), "PV Exported to Grid (kWh)"),
        (lambda df: get_with_suffix(df, "PV.electric_to_load_series_kw", ""), "PV Serving Load (kWh)"),
        (lambda df: get_with_suffix(df, "Wind.annual_energy_produced_kwh", ""), "Wind Total Electricity Produced (kWh)"),
        (lambda df: get_with_suffix(df, "Wind.electric_to_grid_series_kw", ""), "Wind Exported to Grid (kWh)"),
        (lambda df: get_with_suffix(df, "Wind.electric_to_load_series_kw", ""), "Wind Serving Load (kWh)"),
        (lambda df: get_with_suffix(df, "CHP.annual_electric_energy_produced_kwh", ""), "CHP Total Electricity Produced (kWh)"),
        (lambda df: get_with_suffix(df, "CHP.annual_electric_energy_exported_kwh", ""), "CHP Exported to Grid (kWh)"),
        (lambda df: get_with_suffix(df, "CHP.annual_electric_energy_serving_load_kwh", ""), "CHP Serving Load (kWh)"),
        (lambda df: get_with_suffix(df, "CHP.annual_thermal_energy_serving_load_mmbtu", ""), "CHP Serving Thermal Load (MMBtu)"),
        (lambda df: get_with_suffix(df, "ElectricUtility.annual_energy_supplied_kwh", ""), "Grid Purchased Electricity (kWh)"),
        (lambda df: get_with_suffix(df, "ElectricUtility.electric_to_load_series_kw", ""), "Total Site Electricity Use (kWh)"),
        (lambda df: get_with_suffix(df, "ElectricUtility.electric_to_load_series_kwsdf", ""), "Net Purchased Electricity Reduction (%)"),
        (lambda df: get_with_suffix(df, "ElectricTariff.year_one_energy_cost_before_tax", ""), "Electricity Energy Cost ($)"),
        (lambda df: get_with_suffix(df, "ElectricTariff.year_one_demand_cost_before_tax", ""), "Electricity Demand Cost ($)"),
        (lambda df: get_with_suffix(df, "ElectricTariff.year_one_fixed_cost_before_tax", ""), "Utility Fixed Cost ($)"),
        (lambda df: get_with_suffix(df, "ElectricTariff.year_one_bill_before_tax", ""), "Purchased Electricity Cost ($)"),
        (lambda df: get_with_suffix(df, "ElectricTariff.year_one_export_benefit_before_tax", ""), "Electricity Export Benefit ($)"),
        (lambda df: get_with_suffix(df, "ElectricTariff.lifecycle_energy_cost_after_tax", ""), "Net Electricity Cost ($)"),
        (lambda df: get_with_suffix(df, "ElectricTariff.lifecycle_energy_cost_after_tax_bau", ""), "Electricity Cost Savings ($/year)"),
        (lambda df: get_with_suffix(df, "Boiler.fuel_used_mmbtu", ""), "Boiler Fuel (MMBtu)"),
        (lambda df: get_with_suffix(df, "CHP.fuel_used_mmbtu", ""), "CHP Fuel (MMBtu)"),
        (lambda df: get_with_suffix(df, "ElectricUtility.total_energy_supplied_kwh", ""), "Total Fuel (MMBtu)"),
        (lambda df: get_with_suffix(df, "ElectricUtility.annual_energy_supplied_kwh_bau", ""), "Natural Gas Reduction (%)"),
        (lambda df: get_with_suffix(df, "Boiler.annual_thermal_production_mmbtu", ""), "Boiler Thermal Production (MMBtu)"),
        (lambda df: get_with_suffix(df, "CHP.annual_thermal_production_mmbtu", ""), "CHP Thermal Production (MMBtu)"),
        (lambda df: get_with_suffix(df, "CHP.annual_thermal_energy_serving_load_mmbtu", ""), "Total Thermal Production (MMBtu)"),
        (lambda df: get_with_suffix(df, "Site.heating_system_fuel_cost_us_dollars", ""), "Heating System Fuel Cost ($)"),
        (lambda df: get_with_suffix(df, "CHP.year_one_fuel_cost_before_tax", ""), "CHP Fuel Cost ($)"),
        (lambda df: get_with_suffix(df, "Site.total_fuel_cost_us_dollars", ""), "Total Fuel (NG) Cost ($)"),
        (lambda df: get_with_suffix(df, "Site.total_utility_cost_us_dollars", ""), "Total Utility Cost ($)"),
        (lambda df: get_with_suffix(df, "Financial.om_and_replacement_present_cost_after_tax", ""), "O&M Cost Increase ($)"),
        (lambda df: get_with_suffix(df, "Financial.simple_payback_years", ""), "Payback Period (years)"),
        (lambda df: get_with_suffix(df, "Financial.lifecycle_capital_costs", ""), "Gross Capital Cost ($)"),
        (lambda df: get_with_suffix(df, "Financial.total_incentives_us_dollars", ""), "Federal Tax Incentive (30%)"),
        (lambda df: get_with_suffix(df, "Financial.iac_grant_us_dollars", ""), "IAC Grant ($)"),
        (lambda df: get_with_suffix(df, "Financial.total_incentives_value_us_dollars", ""), "Incentive Value ($)"),
        (lambda df: get_with_suffix(df, "Financial.net_capital_cost_us_dollars", ""), "Net Capital Cost ($)"),
        (lambda df: get_with_suffix(df, "Financial.annual_cost_savings_us_dollars", ""), "Annual Cost Savings ($)"),
        (lambda df: get_with_suffix(df, "Financial.simple_payback_years", ""), "Simple Payback (years)"),
        (lambda df: get_with_suffix(df, "Site.annual_emissions_tonnes_CO2", ""), "CO2 Emissions (tonnes)"),
        (lambda df: get_with_suffix(df, "Site.lifecycle_emissions_tonnes_CO2", ""), "CO2 Reduction (tonnes)"),
        (lambda df: get_with_suffix(df, "Site.lifecycle_emissions_tonnes_CO2_bau", ""), "CO2 % savings "),
        (lambda df: get_with_suffix(df, "Financial.npv", ""), "NPV"),
    ]

    bau_values = get_bau_values(scenarios, config)
    combined_df = pd.DataFrame()
    for scenario in scenarios:
        run_uuid = scenario['run_uuid']
        df_result = get_REopt_data(scenario["outputs"], run_uuid, config)
        df_result = df_result.set_index('Scenario').T
        df_result.columns = [run_uuid]
        combined_df = df_result if combined_df.empty else combined_df.join(df_result, how='outer')

    bau_data = {key: [value] for key, value in bau_values.items()}
    bau_data["Scenario"] = ["BAU"]
    df_bau = pd.DataFrame(bau_data)

    combined_df = pd.concat([df_bau, combined_df.T]).reset_index(drop=True)
    combined_df = clean_data_dict(combined_df.to_dict(orient="list"))
    combined_df = pd.DataFrame(combined_df)
    combined_df = combined_df[["Scenario"] + [col for col in combined_df.columns if col != "Scenario"]]

    return combined_df

def summary_by_runuuids(run_uuids):
    """Fetch summary for multiple run UUIDs."""
    if not run_uuids:
        return {'Error': 'Must provide one or more run_uuids'}

    for r_uuid in run_uuids:
        if not isinstance(r_uuid, str):
            return {'Error': f'Provided run_uuids type error, must be string. {r_uuid}'}
        
        try:
            uuid.UUID(r_uuid)
        except ValueError as e:
            return {"Error": str(e)}
    
    try:
        scenarios = [get_scenario_results(run_uuid) for run_uuid in run_uuids]
        return {'scenarios': scenarios}
    except Exception as e:
        return {"Error": str(e)}

### Main Script Execution (`main`)

- **Retrieves the Scenario Data**: Retrieves results from the REopt API for each scenario.
- **Data Processing**: Processes the scenarios using the functions outlined above.
- **Output Handling**: Converts the processed data into JSON format, and provides options to save it as a CSV file.

In [11]:
def main():
    run_uuid_1 = "b620d505-3891-47e4-9be2-b7f150a6dd82"
    run_uuid_2 = "9e7b3977-f3a0-4d2b-b371-392e34b34c50"

    run_uuids = [run_uuid_1, run_uuid_2]

    scenarios = summary_by_runuuids(run_uuids)
    if 'scenarios' not in scenarios:
        print(f"Error: {scenarios['Error']}")
        return

    final_df = process_scenarios(scenarios['scenarios'])
    final_df.iloc[1:, 0] = run_uuids
    
    display(final_df)

    def colnum_string(n):
        """Convert a column number to an Excel-style column string."""
        string = ""
        while n > 0:
            n, remainder = divmod(n - 1, 26)
            string = chr(65 + remainder) + string
        return string

    def create_proforma_with_dataframe(df):
        # Create a new Excel file and add a worksheet
        workbook = xlsxwriter.Workbook('ITA_report.xlsx')
        worksheet = workbook.add_worksheet('ITA Report Template')

        # Write headers
        for col_num, header in enumerate(df.columns):
            worksheet.write(0, col_num, header)
                
        # Write data
        for row_num, row_data in df.iterrows():
            for col_num, value in enumerate(row_data):
                if pd.isnull(value) or value == '-':
                    worksheet.write(row_num + 1, col_num, "")
                else:
                    worksheet.write(row_num + 1, col_num, value)

        # Map column headers to indices
        headers = {header: col_num for col_num, header in enumerate(df.columns)}

        # Fetch the columns indices
        total_site_use_col = headers['Total Site Electricity Use (kWh)']
        pv_to_load = headers['PV Serving Load (kWh)']
        wind_to_load = headers['Wind Serving Load (kWh)']
        chp_to_load = headers['CHP Serving Load (kWh)']
        grid_to_load = headers['Grid Purchased Electricity (kWh)']
        net_elec_reduction_col = headers['Net Purchased Electricity Reduction (%)']
        purchased_elec_costs_col = headers['Purchased Electricity Cost ($)']
        energy_costs = headers['Electricity Energy Cost ($)']
        demand_costs = headers['Electricity Demand Cost ($)']
        fixed_costs = headers['Utility Fixed Cost ($)']
        net_elec_costs_col = headers['Net Electricity Cost ($)']
        export_benefit = headers['Electricity Export Benefit ($)']
        elec_savings_per_year_col = headers['Electricity Cost Savings ($/year)']
        total_fuel_col = headers['Total Fuel (MMBtu)']
        boiler_fuel = headers['Boiler Fuel (MMBtu)']
        chp_fuel = headers['CHP Fuel (MMBtu)']
        ng_reduction_col = headers['Natural Gas Reduction (%)']
        total_therm_col = headers['Total Thermal Production (MMBtu)']
        boiler_prod_mmbtu = headers['Boiler Thermal Production (MMBtu)']
        chp_prod_mmbtu = headers['CHP Thermal Production (MMBtu)']
        ng_costs_col = headers['Total Fuel (NG) Cost ($)']
        heat_fuel_costs = headers['Heating System Fuel Cost ($)']
        chp_fuel_costs = headers['CHP Fuel Cost ($)']
        total_util_cost_col = headers['Total Utility Cost ($)']
        incentive_col = headers['Incentive Value ($)']
        gross_cap_cost = headers['Gross Capital Cost ($)']
        fed_incentive = headers['Federal Tax Incentive (30%)']
        iac_grant     = headers['IAC Grant ($)']
        net_cap_cost_col = headers['Net Capital Cost ($)']
        annual_cost_save_col = headers['Annual Cost Savings ($)']
        om_increase_save_col = headers['O&M Cost Increase ($)']
        simple_payback_col = headers['Simple Payback (years)']
        co2_reduction_col = headers['CO2 Reduction (tonnes)']
        co2_emission_col = headers['CO2 Emissions (tonnes)']
        co2_savings_col = headers['CO2 % savings ']
        
        
        # Assuming the BAU (Business As Usual) row is the first data row after the header
        bau_grid_value_cell = f'{colnum_string(grid_to_load + 1)}2'
        bau_net_cost_value_cell = f'{colnum_string(net_elec_costs_col + 1)}2'
        bau_ng_reduction_value_cell = f'{colnum_string(total_fuel_col + 1)}2'
        bau_util_cost_value_cell = f'{colnum_string(total_util_cost_col + 1)}2'
        bau_co2_reduction_value_cell = f'{colnum_string(co2_emission_col + 1)}2'

        # Write formulas for each row
        for row in range(1, len(df) + 1):
            worksheet.write_formula(row, total_site_use_col, 
                                    f'={colnum_string(pv_to_load + 1)}{row + 1}+{colnum_string(wind_to_load + 1)}{row + 1}+'
                                    f'{colnum_string(chp_to_load + 1)}{row + 1}+{colnum_string(grid_to_load + 1)}{row + 1}')
            
            worksheet.write_formula(row, net_elec_reduction_col,
                                    f'=({bau_grid_value_cell}-{colnum_string(grid_to_load + 1)}{row + 1})/{bau_grid_value_cell}')

            worksheet.write_formula(row, purchased_elec_costs_col, 
                                    f'={colnum_string(energy_costs + 1)}{row + 1}+{colnum_string(demand_costs + 1)}{row + 1}+'
                                    f'{colnum_string(fixed_costs + 1)}{row + 1}')

            worksheet.write_formula(row, net_elec_costs_col, 
                                    f'={colnum_string(purchased_elec_costs_col + 1)}{row + 1}-{colnum_string(export_benefit + 1)}{row + 1}')

            worksheet.write_formula(row, elec_savings_per_year_col,
                                    f'={bau_net_cost_value_cell}-{colnum_string(net_elec_costs_col + 1)}{row + 1}')
            
            worksheet.write_formula(row, total_fuel_col,
                                    f'={colnum_string(boiler_fuel + 1)}{row + 1}+{colnum_string(chp_fuel + 1)}{row + 1}')
            
            worksheet.write_formula(row, ng_reduction_col,
                                    f'=({bau_ng_reduction_value_cell}-{colnum_string(total_fuel_col + 1)}{row + 1})/{bau_ng_reduction_value_cell}')
            
            worksheet.write_formula(row, total_therm_col,
                                    f'={colnum_string(boiler_prod_mmbtu + 1)}{row + 1}+{colnum_string(chp_prod_mmbtu + 1)}{row + 1}')
            
            worksheet.write_formula(row, ng_costs_col,
                                    f'={colnum_string(heat_fuel_costs + 1)}{row + 1}+{colnum_string(chp_fuel_costs + 1)}{row + 1}')
            
            worksheet.write_formula(row, total_util_cost_col,
                                    f'={colnum_string(net_elec_costs_col + 1)}{row + 1}+{colnum_string(ng_costs_col + 1)}{row + 1}')
            
            worksheet.write_formula(row, incentive_col,
                                    f'={colnum_string(fed_incentive + 1)}{row + 1}*{colnum_string(gross_cap_cost + 1)}{row + 1}+{colnum_string(iac_grant + 1)}{row + 1}')
            
            worksheet.write_formula(row, net_cap_cost_col, 
                                    f'={colnum_string(gross_cap_cost + 1)}{row + 1}-{colnum_string(incentive_col + 1)}{row + 1}')
            
            worksheet.write_formula(row, annual_cost_save_col,
                                    f'={bau_util_cost_value_cell}-{colnum_string(total_util_cost_col + 1)}{row + 1}+{colnum_string(om_increase_save_col + 1)}{row + 1}')

            worksheet.write_formula(row, simple_payback_col, 
                                    f'={colnum_string(net_cap_cost_col + 1)}{row + 1}/{colnum_string(annual_cost_save_col + 1)}{row + 1}')
            
            worksheet.write_formula(row, co2_reduction_col,
                                    f'={bau_co2_reduction_value_cell}-{colnum_string(co2_emission_col + 1)}{row + 1}')
            
            worksheet.write_formula(row, co2_savings_col,
                        f'=({bau_co2_reduction_value_cell}-{colnum_string(co2_emission_col + 1)}{row + 1})/{bau_co2_reduction_value_cell}')
            

        # Save the workbook
        workbook.close()
        print("Excel file created: ITA_report.xlsx")

    create_proforma_with_dataframe(final_df)

main()


,Scenario,PV Size (kW),Wind Size (kW),CHP Size (kW),PV Total Electricity Produced (kWh),PV Exported to Grid (kWh),PV Serving Load (kWh),Wind Total Electricity Produced (kWh),Wind Exported to Grid (kWh),Wind Serving Load (kWh),...,Federal Tax Incentive (30%),IAC Grant ($),Incentive Value ($),Net Capital Cost ($),Annual Cost Savings ($),Simple Payback (years),CO2 Emissions (tonnes),CO2 Reduction (tonnes),CO2 % savings,NPV
0,BAU,,,,,,,,,,...,,,,,,,4.200,84.00,,
1,b620d505-3891-47e4-9be2-b7f150a6dd82,32.8409,,,58151.95,,52959.012,,,,...,,,,,,5.42,3.309,66.18,84.0,26725.63
2,9e7b3977-f3a0-4d2b-b371-392e34b34c50,45.2187,,,80069.28,,59921.11,,,,...,,,,,,5.9,2.353,47.06,84.0,37100.54


Excel file created: ITA_report.xlsx
